# Признаки волатильности

- Реализованная волатильность: rolling windows на log_returns
- EWMA-волатильность: экспоненциальное сглаживание
- Parkinson volatility: расчёт на high-low
- Garman-Klass volatility: расчёт на OHLC
- Направленная волатильность: отдельные ап/даун движения

Все расчёты сохраняются структурированно для одного тикера и для всего набора.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Загрузка данных

Используем данные из предыдущего этапа `01_data_loading.ipynb`


In [ ]:
# Загрузка обработанных данных
DATA_DIR = Path('data') / 'processed'

# Пример загрузки для одного тикера
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
print(f"Загружено: {len(df)} записей для {ticker}")
print(df.head())


## Функции расчета волатильности


In [ ]:
def realized_volatility(returns, window=30):
    """Реализованная волатильность: rolling std на log returns"""
    return returns.rolling(window=window).std() * np.sqrt(252)


def ewma_volatility(returns, span=30):
    """EWMA волатильность: экспоненциальное сглаживание"""
    return returns.ewm(span=span).std() * np.sqrt(252)


def parkinson_volatility(high, low, window=30):
    """Parkinson volatility: на основе high-low"""
    hl_ratio = np.log(high / low) ** 2
    return np.sqrt(hl_ratio.rolling(window=window).mean() / (4 * np.log(2))) * np.sqrt(252)


def garman_klass_volatility(open_price, high, low, close, window=30):
    """Garman-Klass volatility: использует OHLC"""
    hl = np.log(high / low) ** 2
    co = np.log(close / open_price) ** 2
    gk = 0.5 * hl - (2 * np.log(2) - 1) * co
    return np.sqrt(gk.rolling(window=window).mean()) * np.sqrt(252)


def directional_volatility(returns, window=30):
    """Направленная волатильность: отдельно вверх и вниз"""
    up_vol = returns[returns > 0].rolling(window=window).std() * np.sqrt(252)
    down_vol = returns[returns < 0].rolling(window=window).std() * np.sqrt(252)
    return up_vol, down_vol

print("✅ Функции волатильности загружены")


## Расчет признаков волатильности


In [ ]:
# Расчет всех типов волатильности
windows = [10, 30, 60]

for window in windows:
    df[f'realized_vol_{window}'] = realized_volatility(df['log_return'], window=window)
    df[f'ewma_vol_{window}'] = ewma_volatility(df['log_return'], span=window)
    df[f'parkinson_vol_{window}'] = parkinson_volatility(df['high'], df['low'], window=window)
    df[f'gk_vol_{window}'] = garman_klass_volatility(df['open'], df['high'], df['low'], df['close'], window=window)

print("✅ Признаки волатильности рассчитаны")
print(f"\nСтолбцы: {df.columns.tolist()}")
print(f"\nПример данных:")
print(df[['date', 'close', 'realized_vol_30', 'ewma_vol_30', 'parkinson_vol_30']].tail())


## Сохранение результатов


In [ ]:
# Сохранение с признаками волатильности
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_volatility_features.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
